In [ ]:
import sys
import os

# Add the project root to the Python path
package_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
print(f"Adding package root to sys.path: {package_root}")
if package_root not in sys.path:
	sys.path.append(package_root)


notebook_dir = os.getcwd()
print(f"Current notebook directory: {notebook_dir}")
# change to the directory to the root of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
print(f"Project root: {project_root}")
os.chdir(project_root)

from dotenv import load_dotenv
load_dotenv()

In [ ]:
print(os.environ["LANGCHAIN_TRACING_V2"])

In [ ]:
from langchain_core.messages import HumanMessage
from lets_talk.utils import get_message_text
import importlib
import nest_asyncio
nest_asyncio.apply()

In [ ]:
#display marked down
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#printmd(r.content)

In [ ]:
from langchain_core.runnables import RunnableConfig
config = RunnableConfig(
        configurable={"user_id": "abc","thread_id":"abc-1" }
    )

In [ ]:
from lets_talk import agent
importlib.reload(agent)
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

uncompiled_graph = agent.build_graph()
graph =  uncompiled_graph.compile(checkpointer=checkpointer)
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response =  await graph.ainvoke({"messages":[HumanMessage(content="I'm Isa, who are you?")] },config=config)
printmd(response["messages"][-1].content)

In [ ]:
response =  await graph.ainvoke({"messages":[HumanMessage(content="Who am I?")] },config=config)
printmd(response["messages"][-1].content)

In [ ]:
response

In [ ]:
response =  await graph.ainvoke({"messages":[HumanMessage(content="I don't like you?")] },config=config)
printmd(response["messages"][-1].content)

In [ ]:
# Update the config dictionary to include recursion_limit
config_with_recursion = {**config, "recursion_limit": 10}

query = "my email is isa@thedataguy.pro, send a message to thedataguy that I like your blog."

response = await graph.ainvoke({"question": query,"messages":[HumanMessage(content=query)]}, config=config_with_recursion)
printmd(response["messages"][-1].content)

In [ ]:
response

In [ ]:
response = await graph.ainvoke({"question": "What is current time in CDT?"},config=config)

printmd(response["messages"][-1].content)


In [ ]:
response

In [ ]:
response = await graph.ainvoke({"question": "What is current time in CDT?"},config=config)

In [ ]:
async for chunk,_ in graph.astream({"messages":[HumanMessage(content="Give me latest blog posts")]}, stream_mode="messages", config=config):
    print(chunk.content,end="")

In [ ]:

import os
import getpass

os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter your Together API key: ")

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
checkpointer = InMemorySaver()

config = RunnableConfig(
        configurable={"user_id": "abc","thread_id":"abc-1" }
)

In [ ]:
import lets_talk.tools as lets_talk_tools

importlib.reload(lets_talk_tools)

from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from lets_talk import rag
from lets_talk.utils import format_docs
from langchain.chat_models import init_chat_model
from langchain_community.tools.requests.tool import RequestsGetTool
from langchain_community.utilities.requests import TextRequestsWrapper


@tool 
def retrive_documents(query: str) -> str:
    """Retrieve relevant documents from the knowledge base to answer user questions.
    
    Use this tool when you need to search for specific information, facts, or content
    that may be in the document collection. Provide a clear search query related to
    what information you need to find.
    
    Args:
        query: The search query to find relevant documents
        
    Returns:
        Formatted text containing the retrieved document content
    """
    docs = rag.retriever.invoke(query) 
    return format_docs(docs)


prompt = """
You are TheDataGuy Chat, a specialized assistant powered by content from Muhammad Afzaal (TheDataGuy)'s blog at thedataguy.pro. You are expert in data science, AI evaluation, RAG systems, research agents, and metric-driven development.

## Your Purpose
You provide practical, insightful responses to queries about topics covered in TheDataGuy's blog posts, including:
- RAGAS and evaluation frameworks for LLM applications
- RAG (Retrieval-Augmented Generation) systems and their implementation
- Building and evaluating AI research agents
- Metric-Driven Development for technology projects
- Data strategy and its importance for business success
- Technical concepts in AI, LLM applications, and data science

## Tools Usage
- Always use the 'retrive_documents' tool when you need to search for information from blog posts or articles
- Use this tool before answering questions about specific content, examples, or details from TheDataGuy's blog
- When using the retrieval tool, provide clear and specific search queries related to the user's question

## Response Guidelines
1. Generate clear, concise responses in markdown format
2. Include relevant links to blog posts to help users find more information
3. For code examples, use appropriate syntax highlighting
4. When practical, provide actionable steps or implementations
5. Maintain a helpful, informative tone consistent with TheDataGuy's writing style
6. When providing links, use the URL format from the context: [title or description](URL)
7. When discussing a series of blog posts, mention related posts when appropriate
8. When faced with rude queries or negative comments, respond with graceful, upbeat positivity and redirect the conversation toward helpful topics

## Special Cases
- If the context is unrelated to the query, respond with "I don't know" and suggest relevant topics that are covered in the blog
- If asked about topics beyond the blog's scope, politely explain your focus areas and suggest checking thedataguy.pro for the latest content
- Use real-world examples to illustrate complex concepts, similar to those in the blog posts
- For rude or impolite queries, maintain a positive and professional tone, never responding with rudeness, and gently steer the conversation back to productive topics

Remember, your goal is to help users understand TheDataGuy's insights and apply them to their own projects and challenges, always maintaining a helpful and positive attitude regardless of how the query is phrased.
"""

requests_tool = RequestsGetTool(
    requests_wrapper=TextRequestsWrapper(headers={}),
    allow_dangerous_requests=True,
    description="Use this tool to make HTTP GET requests to retrieve information from the web. Provide a valid URL to fetch data.",
)

tools =[lets_talk_tools.RSSFeedTool(), lets_talk_tools.get_current_datetime,retrive_documents,requests_tool]

model_name = "openai:gpt-4o-mini"
#model_name = "together:meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
#model_name = "ollama:cogito"
#model_name = "together:meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
model = init_chat_model(model_name, temperature=0.0)

agent = create_react_agent(
    model=model,
    tools=tools,
    prompt=prompt,
    version="v2",
    checkpointer=checkpointer
    
)

# Run the agent


In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Explain thedataguy data value chain?"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "I'm Isa, who are you?"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "What is my name?"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "What is thedataguy's data value chain?"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Who are you?"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Who is thedataguy?"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "What is latest blog post?"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Explain Truth is Cold by fetching contents"}]},config=config
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Summarize https://thedataguy.pro/blog/2025/05/truth-is-cold"}]},config=config
)



In [ ]:
state = agent.get_state(config=config)

In [ ]:
for m in state.values["messages"]:
    m.pretty_print()

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Explain lastest post"}]},config=config
)

